In [13]:
import numpy as np
print(np.argmax(np.bincount([0,0,1],weights=[0.2,0.2,0.6])))

ex = np.array([[0.9,0.1],
               [0.8,0.2],
               [0.4,0.6]])
p = np.average(ex, axis=0,weights=[0.2,0.2,0.6])
print(p)
print(np.argmax(p))

1
[ 0.58  0.42]
0


In [14]:
# お試し
print(np.bincount([0,0,2]))
print(np.bincount([0,0,1],weights=[0.2,0.2,0.6]))
print(np.bincount([0,0,2,2],weights=[0.2,0.2,0.6,0.5]))

[2 0 1]
[ 0.4  0.6]
[ 0.4  0.   1.1]
